In [1]:
import pandas as pd
import numpy as np

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.svm import SVC 

import warnings
warnings.filterwarnings("ignore")

In [2]:
data_train = pd.read_csv('data_train_nostopword.csv', sep=';')
data_test = pd.read_csv('data_test_nostopword.csv', sep=';')

In [3]:
data_train.head()

,review_text,price,packaging,product,aroma
0,dair dulu penasaran banget sama skincare ini k...,-1,0,0.0,0
1,dari awal ngincer ini karena banyak yang bilan...,0,0,1.0,0
2,aku suka sekali tabirsurya ini tekstur ringan ...,0,1,1.0,0
3,pernah nyobain ini pas muka lagi stress ny dan...,0,0,-1.0,0
4,paling suka sama gokujyun losion untuk merek h...,1,0,1.0,0


In [4]:
data_test.head()

,review_text,price,packaging,product,aroma
0,produk favorit banget enak dipake plus tidak p...,-1,0,1.0,0
1,awal agak lengket pas baru dipake tapi untuk h...,1,0,0.0,0
2,minyak kontrol lumayan bisa ngempesin jerawat ...,1,0,-1.0,0
3,favorite banget selain harga terjangkau banget...,1,0,1.0,0
4,ini adalah exfoliant toner yang saya pakai bag...,0,0,1.0,0


In [5]:
data_train.isnull().sum()

review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [6]:
data_test.isnull().sum()

review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [7]:
X_train = data_train['review_text']
X_test = data_test['review_text']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
fitur_train = TfidfVectorizer().fit(X_train)
tfidf_train = fitur_train.transform(X_train).toarray()
tfidf_test = fitur_train.transform(X_test).toarray()

In [9]:
def get_fitur_tfidf(tfidf, fitur):
    terms = tfidf.get_feature_names()
    sums = fitur.sum(axis = 0)

    data = []
    for col, term in enumerate(terms):
        data.append((term, sums[col] ))
    ranking = pd.DataFrame(data, columns=['Terms','TF-IDF'])
    return ranking

In [10]:
out_tfidf_tr = get_fitur_tfidf(fitur_train, tfidf_train)

In [11]:
out_tfidf_tr.shape

(7540, 2)

In [12]:
#FEATURE SELECTION CHI SQUARE
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [13]:
x_chi_tr = tfidf_train
x_chi_test = tfidf_test

In [14]:
#y_chi_tr = np.array(data_train[['price','packaging','aroma','product']])
#y_chi_test = np.array(data_test[['price','packaging','aroma','product']])

In [15]:
y_price_tr = (data_train['price'])
y_packaging_tr = (data_train['packaging'])
y_product_tr = (data_train['product'])
y_aroma_tr = (data_train['aroma'])

In [16]:
y_price_test = (data_test['price'])
y_packaging_test = (data_test['packaging'])
y_product_test = (data_test['product'])
y_aroma_test = (data_test['aroma'])

In [17]:
chi_price_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_price_tr)
chi_packaging_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_packaging_tr)
chi_product_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_product_tr)
chi_aroma_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_aroma_tr)


chi_price_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_price_test)
chi_packaging_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_packaging_test)
chi_product_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_product_test)
chi_aroma_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_aroma_test)

In [18]:
from sklearn.neighbors import KNeighborsClassifier

#aspek price
knn_price_7 = KNeighborsClassifier(n_neighbors=50)


knn_price_7.fit(chi_price_tr, y_price_tr)


predictions_price_7 = knn_price_7.predict(chi_price_test)

#aspek packaging

knn_packaging_7 = KNeighborsClassifier(n_neighbors=50)

knn_packaging_7.fit(chi_packaging_tr, y_packaging_tr)

predictions_packaging_7 = knn_packaging_7.predict(chi_packaging_test)

#aspek product

knn_product_7 = KNeighborsClassifier(n_neighbors=50)



knn_product_7.fit(chi_product_tr, y_product_tr)



predictions_product_7 = knn_product_7.predict(chi_product_test)

#aspek aroma

knn_aroma_7 = KNeighborsClassifier(n_neighbors=50)


knn_aroma_7.fit(chi_aroma_tr, y_aroma_tr)


predictions_aroma_7 = knn_aroma_7.predict(chi_aroma_test)


In [19]:
#Nilai price

knn7_price_skor=knn_price_7.score(chi_price_test,y_price_test)

#nilai packaging

knn7_packaging_skor=knn_packaging_7.score(chi_packaging_test,y_packaging_test)

#nilai product

knn7_product_skor=knn_product_7.score(chi_product_test,y_product_test)

#nilai aroma

knn7_aroma_skor=knn_aroma_7.score(chi_aroma_test,y_aroma_test)

print('=============== PRICE ===================')

print('Nilai Model price K-NN pada K-50           :',knn7_price_skor * 100 ,'%')

print('                                                                             ')
print('============= PACKAGING ================')

print('Nilai Model packaging K-NN pada K-50       :',knn7_packaging_skor * 100 ,'%')

print('                                                                             ')
print('============= PRODUCT ================')

print('Nilai Model product K-NN pada K-50         :',knn7_product_skor * 100 ,'%')

print('                                                                             ')
print('============= AROMA ================')

print('Nilai Model aroma K-NN pada K-50           :',knn7_aroma_skor * 100 ,'%')


=============== PRICE ===================
Nilai Model price K-NN pada K-50           : 57.1969696969697 %
                                                                             
============= PACKAGING ================
Nilai Model packaging K-NN pada K-50       : 84.72222222222221 %
                                                                             
============= PRODUCT ================
Nilai Model product K-NN pada K-50         : 67.8030303030303 %
                                                                             
============= AROMA ================
Nilai Model aroma K-NN pada K-50           : 79.54545454545455 %


In [20]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
#from sklearn.metrics import  multilabel_confusion_matrix

In [21]:
#Evaluasi aspek Price Pada K = 50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_price_test,predictions_price_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_price_test,predictions_price_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_price_test,predictions_price_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_price_test,predictions_price_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_price_test,predictions_price_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_price_test,predictions_price_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_price_test,predictions_price_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_price_test,predictions_price_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_price_test,predictions_price_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_price_test,predictions_price_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_price_test,predictions_price_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_price_test,predictions_price_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  0 139  13]
 [  0 407  12]
 [  4 171  46]]

Accuracy: 0.57

Micro Precision: 0.57
Micro Recall: 0.57
Micro F1-score: 0.57

Macro Precision: 0.41
Macro Recall: 0.39
Macro F1-score: 0.34

Weighted Precision: 0.48
Weighted Recall: 0.57
Weighted F1-score: 0.47

Classification Report

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       152
           0       0.57      0.97      0.72       419
           1       0.65      0.21      0.32       221

    accuracy                           0.57       792
   macro avg       0.41      0.39      0.34       792
weighted avg       0.48      0.57      0.47       792



In [22]:
#Evaluasi aspek Packaging pada K= 50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_packaging_test,predictions_packaging_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_packaging_test,predictions_packaging_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_packaging_test,predictions_packaging_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_packaging_test,predictions_packaging_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_packaging_test,predictions_packaging_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_packaging_test,predictions_packaging_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_packaging_test,predictions_packaging_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_packaging_test,predictions_packaging_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_packaging_test,predictions_packaging_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_packaging_test,predictions_packaging_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_packaging_test,predictions_packaging_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_packaging_test,predictions_packaging_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  0  39   2]
 [  0 670   2]
 [  0  78   1]]

Accuracy: 0.85

Micro Precision: 0.85
Micro Recall: 0.85
Micro F1-score: 0.85

Macro Precision: 0.35
Macro Recall: 0.34
Macro F1-score: 0.31

Weighted Precision: 0.74
Weighted Recall: 0.85
Weighted F1-score: 0.78

Classification Report

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        41
           0       0.85      1.00      0.92       672
           1       0.20      0.01      0.02        79

    accuracy                           0.85       792
   macro avg       0.35      0.34      0.31       792
weighted avg       0.74      0.85      0.78       792



In [23]:
#Evaluasi aspek Product pada K= 50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_product_test,predictions_product_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_product_test,predictions_product_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_product_test,predictions_product_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_product_test,predictions_product_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_product_test,predictions_product_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_product_test,predictions_product_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_product_test,predictions_product_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_product_test,predictions_product_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_product_test,predictions_product_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_product_test,predictions_product_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_product_test,predictions_product_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_product_test,predictions_product_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  3   3 129]
 [  2   0 119]
 [  0   2 534]]

Accuracy: 0.68

Micro Precision: 0.68
Micro Recall: 0.68
Micro F1-score: 0.68

Macro Precision: 0.43
Macro Recall: 0.34
Macro F1-score: 0.28

Weighted Precision: 0.56
Weighted Recall: 0.68
Weighted F1-score: 0.56

Classification Report

              precision    recall  f1-score   support

          -1       0.60      0.02      0.04       135
           0       0.00      0.00      0.00       121
           1       0.68      1.00      0.81       536

    accuracy                           0.68       792
   macro avg       0.43      0.34      0.28       792
weighted avg       0.56      0.68      0.56       792



In [24]:
#Evaluasi aspek Aroma Pada K=50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_aroma_test,predictions_aroma_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_aroma_test,predictions_aroma_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_aroma_test,predictions_aroma_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_aroma_test,predictions_aroma_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_aroma_test,predictions_aroma_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_aroma_test,predictions_aroma_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_aroma_test,predictions_aroma_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_aroma_test,predictions_aroma_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_aroma_test,predictions_aroma_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_aroma_test,predictions_aroma_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_aroma_test,predictions_aroma_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_aroma_test,predictions_aroma_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  2  35   3]
 [  0 614   1]
 [  0 123  14]]

Accuracy: 0.80

Micro Precision: 0.80
Micro Recall: 0.80
Micro F1-score: 0.80

Macro Precision: 0.86
Macro Recall: 0.38
Macro F1-score: 0.39

Weighted Precision: 0.80
Weighted Recall: 0.80
Weighted F1-score: 0.72

Classification Report

              precision    recall  f1-score   support

          -1       1.00      0.05      0.10        40
           0       0.80      1.00      0.89       615
           1       0.78      0.10      0.18       137

    accuracy                           0.80       792
   macro avg       0.86      0.38      0.39       792
weighted avg       0.80      0.80      0.72       792

